In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [2]:
#Estableciendo la conexion con la base de datos aniv_cidade_event en MongoDB
db = MongoClient().aniv_cidade_event

In [3]:
#Punto inicial del omnibus 410 Praça Saens Pena
initial_point = (-22.923736, -43.235721)


In [4]:
#Punto final del omnibus 410 Terminal PUC
final_point = (-22.979408, -43.231352)

In [5]:
inverted_initial_point = (-43.235721, -22.923736)

In [6]:
inverted_final_point = (-43.231352, -22.979408)

In [7]:
#Function to compute the bus_ids circulating during a day 20150208 of bus line 438
def bus_id_list ():
    bus_id_20150215_410 = db.dados20150215_line_410.distinct("bus_id")
    return bus_id_20150215_410
    

In [8]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'A72050',
 u'A72023',
 u'A72129',
 u'A72110',
 u'A72095',
 u'A72065',
 u'A72071',
 u'A72126',
 u'A72152',
 u'A72006',
 u'A72018',
 u'A72055',
 u'A72125',
 u'A72053',
 u'A72174']

In [9]:
len(busId_list)

15

In [10]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId       
    

In [11]:
array_busId = get_busId_list()
array_busId

['A72050',
 'A72023',
 'A72129',
 'A72110',
 'A72095',
 'A72065',
 'A72071',
 'A72126',
 'A72152',
 'A72006',
 'A72018',
 'A72055',
 'A72125',
 'A72053',
 'A72174']

In [15]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150215_line_410.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.00052543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [16]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto initial_point(-22.988199999999999, -43.228299999999997) aqui se pasan a la inversa
sorted_near_initial_points = near_points_to_origin ("A72050", inverted_initial_point)
sorted_near_initial_points

,timestamp,latitude,longitude,speed
0,15-02-2015 05:05:08,-22.923840,-43.235760,12.22
1,15-02-2015 07:16:43,-22.924011,-43.235775,0.00
2,15-02-2015 07:17:53,-22.923910,-43.235790,0.19
3,15-02-2015 07:18:38,-22.923950,-43.235790,1.30
4,15-02-2015 07:19:48,-22.923967,-43.235760,0.56
5,15-02-2015 07:20:23,-22.923857,-43.235790,1.11
6,15-02-2015 07:21:33,-22.923979,-43.235790,0.00
7,15-02-2015 07:28:16,-22.923979,-43.235806,0.00
8,15-02-2015 09:23:09,-22.923986,-43.236084,0.56
9,15-02-2015 09:32:25,-22.923901,-43.235825,0.74


In [17]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos a PUC
#Recordar que aqui las coordenadas del punto se pasan a la inversa(long, lat)
sorted_near_final_points = near_points_to_origin ("A72050", inverted_final_point)
sorted_near_final_points

,timestamp,latitude,longitude,speed
0,15-02-2015 05:54:19,-22.979362,-43.231365,1.48
1,15-02-2015 05:55:47,-22.979424,-43.231365,0.19
2,15-02-2015 05:56:33,-22.979471,-43.231365,0.19
3,15-02-2015 05:58:48,-22.979342,-43.231396,0.19
4,15-02-2015 05:59:48,-22.979589,-43.231380,0.37
5,15-02-2015 08:20:43,-22.979414,-43.231316,0.93
6,15-02-2015 08:21:18,-22.979387,-43.231266,0.74
7,15-02-2015 08:22:49,-22.979496,-43.231365,0.00
8,15-02-2015 08:23:42,-22.979456,-43.231365,0.74
9,15-02-2015 10:40:21,-22.979115,-43.231445,34.63


In [18]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [19]:
#pintando los puntos más cerca del punto inicial
fmap=folium.Map(location=[-22.950635, -43.210436], zoom_start=12)

for row in sorted_near_initial_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='violet', num_sides=4, radius=6)

embed_map(fmap)

In [20]:
#pintando los puntos más cerca del punto final (Vila Isabel)
for row in sorted_near_final_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='red', num_sides=4, radius=6)

embed_map(fmap)

In [21]:
#Pintando Circulos de radio de 1km alrededor de los puntos incial y final 
fmap.simple_marker(final_point, popup='Parada de Vila Isabel')
fmap.simple_marker(initial_point, popup='Parada Leblon')

#Punto inicial
fmap.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap.circle_marker(location=initial_point, radius=50,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap)

In [22]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']
initial_points_timestamp

0     15-02-2015 05:05:08
1     15-02-2015 07:16:43
2     15-02-2015 07:17:53
3     15-02-2015 07:18:38
4     15-02-2015 07:19:48
5     15-02-2015 07:20:23
6     15-02-2015 07:21:33
7     15-02-2015 07:28:16
8     15-02-2015 09:23:09
9     15-02-2015 09:32:25
10    15-02-2015 09:33:35
11    15-02-2015 09:34:45
12    15-02-2015 09:35:20
13    15-02-2015 09:40:28
14    15-02-2015 11:50:24
15    15-02-2015 11:51:43
16    15-02-2015 11:52:53
17    15-02-2015 11:53:22
18    15-02-2015 11:53:56
19    15-02-2015 14:15:34
20    15-02-2015 14:16:45
21    15-02-2015 14:17:54
22    15-02-2015 14:18:50
23    15-02-2015 14:19:33
24    15-02-2015 14:20:43
25    15-02-2015 14:21:53
26    15-02-2015 16:19:50
27    15-02-2015 16:20:26
28    15-02-2015 16:21:35
29    15-02-2015 16:22:45
30    15-02-2015 18:58:52
31    15-02-2015 19:00:37
Name: timestamp, dtype: object

In [23]:
final_points_timestamp = sorted_near_final_points['timestamp']
final_points_timestamp

0     15-02-2015 05:54:19
1     15-02-2015 05:55:47
2     15-02-2015 05:56:33
3     15-02-2015 05:58:48
4     15-02-2015 05:59:48
5     15-02-2015 08:20:43
6     15-02-2015 08:21:18
7     15-02-2015 08:22:49
8     15-02-2015 08:23:42
9     15-02-2015 10:40:21
10    15-02-2015 10:41:44
11    15-02-2015 10:42:55
12    15-02-2015 10:43:30
13    15-02-2015 10:44:40
14    15-02-2015 12:58:52
15    15-02-2015 12:58:53
16    15-02-2015 13:00:39
17    15-02-2015 13:01:49
18    15-02-2015 13:02:24
19    15-02-2015 13:10:27
20    15-02-2015 15:17:47
21    15-02-2015 15:19:46
22    15-02-2015 17:39:19
23    15-02-2015 17:40:53
24    15-02-2015 17:41:43
25    15-02-2015 17:41:59
26    15-02-2015 17:44:56
Name: timestamp, dtype: object

In [24]:
#Funcion que retorna el arreglo con las horas de salida del omnibus al punto inicial/final del trayecto
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    return list_departure_hours

In [25]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
leblon_departure_hours = bus_departure_hour(initial_points_timestamp)
leblon_departure_hours

[datetime.datetime(2015, 2, 15, 5, 5, 8),
 datetime.datetime(2015, 2, 15, 7, 28, 16),
 datetime.datetime(2015, 2, 15, 9, 40, 28),
 datetime.datetime(2015, 2, 15, 11, 53, 56),
 datetime.datetime(2015, 2, 15, 14, 21, 53),
 datetime.datetime(2015, 2, 15, 16, 22, 45),
 datetime.datetime(2015, 2, 15, 19, 0, 37)]

In [26]:
#Llamada a la funcion bus_departure_hour para el punto Vila Isabel  (devuelve todas las horas en que salio el omnibus de Vila Isabel)
vilaIsabel_departure_hours = bus_departure_hour(final_points_timestamp)
vilaIsabel_departure_hours

[datetime.datetime(2015, 2, 15, 5, 59, 48),
 datetime.datetime(2015, 2, 15, 8, 23, 42),
 datetime.datetime(2015, 2, 15, 10, 44, 40),
 datetime.datetime(2015, 2, 15, 13, 10, 27),
 datetime.datetime(2015, 2, 15, 15, 19, 46),
 datetime.datetime(2015, 2, 15, 17, 44, 56)]

In [27]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    if len(initial_points_timestamp_list) >1:
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    return list_arrive_hours

In [28]:
#Llamada a la funcion bus_arrive_hour para el punto Leblon  (devuelve todas las horas en que llegó el omnibus a Leblon)
arrive_hour_to_initialpoint = bus_arrive_hour(initial_points_timestamp)
arrive_hour_to_initialpoint

[datetime.datetime(2015, 2, 15, 5, 5, 8),
 datetime.datetime(2015, 2, 15, 7, 16, 43),
 datetime.datetime(2015, 2, 15, 9, 23, 9),
 datetime.datetime(2015, 2, 15, 11, 50, 24),
 datetime.datetime(2015, 2, 15, 14, 15, 34),
 datetime.datetime(2015, 2, 15, 16, 19, 50),
 datetime.datetime(2015, 2, 15, 18, 58, 52)]

In [29]:
#Llamada a la funcion bus_arrive_hour para el punto Vila Isabel  (devuelve todas las horas en que llegó el omnibus a Vila Isabel)
arrive_hour_to_finalpoint = bus_arrive_hour(final_points_timestamp)
arrive_hour_to_finalpoint

[datetime.datetime(2015, 2, 15, 5, 54, 19),
 datetime.datetime(2015, 2, 15, 8, 20, 43),
 datetime.datetime(2015, 2, 15, 10, 40, 21),
 datetime.datetime(2015, 2, 15, 12, 58, 52),
 datetime.datetime(2015, 2, 15, 15, 17, 47),
 datetime.datetime(2015, 2, 15, 17, 39, 19)]

In [30]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [31]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            direction = name_A + "_to_" + name_B
            departure_hour = departure_array_A[i]
            i = i +1
            arrive_hour = arrive_array_B[j]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
                        
        elif(departure_array_B[j] < departure_array_A[i]):
            direction = name_B + "_to_" + name_A
            departure_hour = departure_array_B[j]
            j = j +1
            arrive_hour = arrive_array_A[i]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  


In [50]:
#dados para llamar a la funcion

mytabla = create_travel_table("Saens Pena", arrive_hour_to_initialpoint, leblon_departure_hours, "PUC", arrive_hour_to_vilaisabel, vilaIsabel_departure_hours, 'A72065', '410' )
mytabla

NameError: name 'arrive_hour_to_leblon' is not defined

In [32]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df
        

        #el problema está en sorted_near_initial_points cuando devuelve vacio para algunos bus_id

In [33]:
query = all_travel_table ('410', inverted_initial_point, 'Saens Pena', inverted_final_point, 'PUC', array_busId)
query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,410,A72050,Saens Pena_to_PUC,2015-02-15 05:05:08,2015-02-15 05:54:19,0:49
1,410,A72050,PUC_to_Saens Pena,2015-02-15 05:59:48,2015-02-15 07:16:43,1:16
2,410,A72050,Saens Pena_to_PUC,2015-02-15 07:28:16,2015-02-15 08:20:43,0:52
3,410,A72050,PUC_to_Saens Pena,2015-02-15 08:23:42,2015-02-15 09:23:09,0:59
4,410,A72050,Saens Pena_to_PUC,2015-02-15 09:40:28,2015-02-15 10:40:21,0:59
5,410,A72050,PUC_to_Saens Pena,2015-02-15 10:44:40,2015-02-15 11:50:24,1:05
6,410,A72050,Saens Pena_to_PUC,2015-02-15 11:53:56,2015-02-15 12:58:52,1:04
7,410,A72050,PUC_to_Saens Pena,2015-02-15 13:10:27,2015-02-15 14:15:34,1:05
8,410,A72050,Saens Pena_to_PUC,2015-02-15 14:21:53,2015-02-15 15:17:47,0:55
9,410,A72050,PUC_to_Saens Pena,2015-02-15 15:19:46,2015-02-15 16:19:50,1:00
